**To run this code you must install all the dependencies in the `docker/requirements.txt` in your personal coputer**

# Cargue de Usuarios a SQL

In [1]:
import pandas as pd
import sqlalchemy

import os
import dotenv

dotenv.load_dotenv()

True

In [28]:
# Configuración de la cadena de conexión
SQL_USER=os.getenv("SQL_USER")
SQL_PASSWORD=os.getenv("SQL_PASSWORD")
SQL_HOST=os.getenv("SQL_HOST")
SQL_PORT=os.getenv("SQL_PORT")
SQL_DATABASE=os.getenv("SQL_DATABASE")

# Cadena de conexión utilizando pyodbc
connection_string = f"mssql+pyodbc://{SQL_USER}:{SQL_PASSWORD}@{SQL_HOST}:{SQL_PORT}/{SQL_DATABASE}?driver=SQL+Server"

# Crear el motor de conexión
engine = sqlalchemy.create_engine(connection_string)

In [29]:
data = pd.read_csv("./data/vendedores_projecto_final_bases_de_datos.csv")
data

,numero_vendedor,username,password,first_name,last_name,title
0,NV-0,carmas,supersecret,Carlos,Masmela,Sales Manager
1,NV-1,yupe,supersecret,Yudy,Pedraza,Sales Executive
2,NV-2,fama,supersecret,Fabian,Madero,VP Sales
3,NV-3,elsa,supersecret,Elkin,Sanchez,EVP Sales
4,NV-4,gial,supersecret,Giovanny,Albarracin,Sales Assistant


In [30]:
data.to_sql('users', con=engine, if_exists='replace', index=False)

-1

# Cargue de Predicciones

In [2]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
import pymongo

import sqlalchemy

import os
import dotenv

dotenv.load_dotenv()

import pandas as pd
import json

import joblib

In [5]:
def read_data_mongodb(USERNAME: str, PASSWORD: str, SERVER: str) -> json:
    """Función que lee los datos de MongoDB

    Args:
        USERNAME (str): Usuario de la base de datos
        PASSWORD (str): Contraseña de la base de datos
        SERVER (str): Servidor de la base de datos

    Returns:
        json: Datos finales
    """

    uri = f"mongodb+srv://{USERNAME}:{PASSWORD}@{SERVER}.czf3nfw.mongodb.net/?retryWrites=true&w=majority&appName={SERVER}"

    # Crear cliente y conectar al servidor
    client = MongoClient(uri, server_api=ServerApi("1"))

    # Enviar PING para conocer si hay conexión  exitosa
    try:
        client.admin.command("ping")
        print("Pinged your deployment. You successfully connected to MongoDB!")
    except Exception as e:
        print(e)

    # Conectarse a la base de datos
    db = client["final_project"]

    # Conectarse a la colección
    collection_oppty = db["opportunities"]

    # Leer datos excluyendo el campo '_id'
    documents = collection_oppty.find(
        {},
        {
            "_id": 0,
        },
    )

    # Convertir a DataFrame
    df = pd.DataFrame(list(documents))

    return df.to_json(orient="records")


def predict_win_or_loose(data) -> json:
    """Predecir si se ganará o se perderá

    Returns:
        json: Datos predicción
    """
    # Tomar la data del paso anterior
    # data = context["task_instance"].xcom_pull(task_ids="read_data_mongodb")
    df = pd.read_json(data, orient="records")

    df = df[["numero_opportunidad",
            "unidad_negocio",
            "etapas",
            "tipos",
            "acv",
            "tcv",
            "margen_ganancia",
            "regiones",
            "territorios",
            "industrias",
            "edad_oferta",
            "lead_source",]]

    # Leer modelo guardado
    model = joblib.load("../model/ml_model.joblib")

    # Predecir si se ganará o se perderá
    y_pred = model.predict(df.drop(columns=["etapas", "numero_opportunidad"]))

    # Transformar solución a cargar a Redis (Numero Oppty y Predicción)
    df_resul = pd.concat([df["numero_opportunidad"], pd.DataFrame(y_pred)], axis=1)
    df_resul.columns = ["numero_opportunidad", "prediccion_modelo"]

    return df_resul.to_json(orient="records")


def save_data_sql_server(
    SQL_USER: str, SQL_PASSWORD: str, SQL_HOST: str, SQL_PORT: str, SQL_DATABASE: str, data, data_inicial #, **context
) -> None:
    """
    Graba datos en redis
    """
    # Tomar la data del paso anterior
    # data = context["task_instance"].xcom_pull(task_ids="predict_win_or_loose")
    df_resul = pd.read_json(data_inicial, orient="records")
    df_prediccion = pd.read_json(data, orient="records")
    df_prediccion=df_prediccion.drop(["numero_opportunidad"], axis=1)
    df_consolidado = pd.concat([df_resul, df_prediccion], axis=1).sort_values(by="numero_opportunidad")
    # Cadena de conexión utilizando pyodbc
    connection_string = f"mssql+pyodbc://{SQL_USER}:{SQL_PASSWORD}@{SQL_HOST}:{SQL_PORT}/{SQL_DATABASE}?driver=SQL+Server"

    # Crear el motor de conexión
    engine = sqlalchemy.create_engine(connection_string)

    # Oportunidad de ventas a SQL
    df_consolidado.to_sql('oportunidades', con=engine, if_exists='replace', index=False)

    return df_consolidado

In [10]:
df_mongodb = read_data_mongodb(USERNAME=os.getenv("MONGODB_USER"), PASSWORD=os.getenv("MONGODB_PASSWORD"), SERVER=os.getenv("MONGODB_SERVER"))
df_prediccion = predict_win_or_loose(df_mongodb)

Pinged your deployment. You successfully connected to MongoDB!


In [11]:
df_consolidado = save_data_sql_server(SQL_USER= os.getenv("SQL_USER"), SQL_PASSWORD= os.getenv("SQL_PASSWORD"), SQL_HOST= os.getenv("SQL_HOST"), SQL_PORT= os.getenv("SQL_PORT"), SQL_DATABASE= os.getenv("SQL_DATABASE"), data_inicial=df_prediccion, data=df_mongodb)
df_consolidado

,numero_opportunidad,prediccion_modelo,cliente,unidad_negocio,etapas,tipos,acv,tcv,margen_ganancia,regiones,territorios,industrias,fecha_creacion_oportunidad,fecha_cierre_oportunidad,edad_oferta,lead_source,numero_vendedor
1,OP-00000,Lost,CL-01169,BU-05,Lost,TY-04,476458.92,943194.18,0.06,RE-02,TE-15,IN-23,2021-08-17,2022-02-06,173,LS-06,NV-4
3,OP-00001,Won,CL-00808,BU-05,Won,TY-00,8715.31,8715.31,0.07,RE-02,TE-01,IN-37,2020-07-23,2020-07-23,0,LS-06,NV-4
5,OP-00002,Lost,CL-00234,BU-05,Lost,TY-04,140992.94,140992.94,0.19,RE-02,TE-15,IN-10,2021-06-21,2021-12-11,173,LS-14,NV-1
2,OP-00003,Won,CL-00808,BU-05,Won,TY-00,1333.11,1333.11,0.19,RE-02,TE-00,IN-55,2020-08-09,2020-08-09,0,LS-11,NV-0
6,OP-00004,Lost,CL-00815,BU-05,Lost,TY-05,388946.05,388946.05,0.28,RE-02,TE-10,IN-46,2021-06-09,2022-11-22,531,LS-28,NV-2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13607,OP-13786,Lost,CL-00679,BU-07,Lost,TY-05,197076.01,197076.01,0.29,RE-03,TE-14,IN-51,2018-03-29,2018-07-31,124,LS-11,NV-4
13366,OP-13787,Won,CL-00679,BU-07,Won,TY-05,227457.25,227457.25,0.27,RE-03,TE-09,IN-39,2018-03-01,2018-03-21,20,LS-04,NV-0
13144,OP-13788,Lost,CL-01239,BU-07,Lost,TY-04,291709.54,388946.05,0.17,RE-03,TE-04,IN-49,2022-08-12,2023-01-15,156,LS-12,NV-4
13277,OP-13789,Lost,CL-01239,BU-07,Lost,TY-04,60286.64,60286.64,0.06,RE-03,TE-12,IN-31,2021-02-04,2021-08-04,181,LS-28,NV-1
